In [391]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn import linear_model
import seaborn as sns
%matplotlib inline

# Get the Data
We'll work with the Automatic Repair of payments.First we will find probability of error, then correct it. Data is provided in excel sheet which has various data like debtor, Bic, Amt, Cheque No etc, We will read data as Data Frame

In [392]:
df = pd.read_csv("Intern.csv",error_bad_lines=False)
df.columns

df.head()
print(df)

     PmtInfId  Dbtr.Nm   Dbtr.ActNo Dbtr.DbtrAgt     Dbtr.BIC  Cdtr.Nm  \
0         Viw    Apple  45831890872  Wells Fargo  WFBIUS6S123  Walmart   
1         5L4  Walmart  54698287911  Wells Fargo  WFBIUS6S123  Walmart   
2         dae    Apple  45831890872  Wells Fargo  WFBIUS6S123     Meta   
3         mRh   Google  16182937803  Wells Fargo  WFBIUS6S123    Apple   
4         2KG    Apple  45831890872  Wells Fargo  WFBIUS6S123   Amazon   
...       ...      ...          ...          ...          ...      ...   
3195      E30     Meta  97631540681  Wells Fargo  WFBIUS6S123   Amazon   
3196      0xg     Meta  97631540681  Wells Fargo  WFBIUS6S123   Amazon   
3197      AF1    Apple  45831890872  Wells Fargo  WFBIUS6S123  Walmart   
3198      G5s    Apple  45831890872  Wells Fargo  WFBIUS6S123    Apple   
3199      4sy    Apple  45831890872  Wells Fargo  WFBIUS6S123  Walmart   

        Cdtr.ActNo    Cdtr.CdtrAgt     Cdtr.BIC  NbOfTxs  ...  PmtMtd  Ccy  \
0       2689301692     Capital On

In [393]:
print('='*50)
print("\nData Information\n")
print('='*50)
df.info()


Data Information

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 21 columns):
PmtInfId        3200 non-null object
Dbtr.Nm         3200 non-null object
Dbtr.ActNo      3200 non-null int64
Dbtr.DbtrAgt    3200 non-null object
Dbtr.BIC        3200 non-null object
Cdtr.Nm         3200 non-null object
Cdtr.ActNo      3200 non-null int64
Cdtr.CdtrAgt    3200 non-null object
Cdtr.BIC        3200 non-null object
NbOfTxs         3200 non-null int64
Amt             3200 non-null float64
PmtMtd          3200 non-null object
Ccy             3200 non-null object
CreDtTm         3200 non-null object
InstrPrty       3200 non-null object
XchgRateInf     28 non-null float64
XchgRate        3200 non-null int64
ChqNb           3200 non-null int64
MemoFld         3200 non-null object
Rsn             1600 non-null object
Class           3200 non-null int64
dtypes: float64(2), int64(6), object(13)
memory usage: 525.1+ KB


In [394]:
# Generate descriptive statistics.
# Checking Min, Max, Mean, Std for all the columns
df.describe(percentiles=[.10, .25, .5, .75])

,Dbtr.ActNo,Cdtr.ActNo,NbOfTxs,Amt,XchgRateInf,XchgRate,ChqNb,Class
count,3.200000e+03,3.200000e+03,3200.0,3200.000000,28.000000,3200.0,3.200000e+03,3200.000000
mean,6.191324e+10,1.020180e+11,1.0,49618.476237,0.428571,1.0,8.670001e+05,2.500000
std,2.896299e+10,1.892186e+11,0.0,28819.045823,0.503953,0.0,2.150658e+06,1.118209
min,1.618294e+10,2.077726e+09,1.0,101.330000,0.000000,1.0,0.000000e+00,1.000000
10%,1.618294e+10,1.232938e+10,1.0,10111.007000,0.000000,1.0,6.000000e+00,1.000000
25%,4.583189e+10,1.892678e+10,1.0,24913.775000,0.000000,1.0,7.250000e+01,1.750000
50%,5.469829e+10,4.583189e+10,1.0,49784.750000,0.000000,1.0,4.894000e+03,2.500000
75%,8.574631e+10,6.892346e+10,1.0,73492.075000,1.000000,1.0,3.019525e+05,3.250000
max,9.763154e+10,9.763154e+11,1.0,99822.340000,1.000000,1.0,9.749684e+06,4.000000


# Checking is there any null values

In [395]:
df.isnull().any()

PmtInfId        False
Dbtr.Nm         False
Dbtr.ActNo      False
Dbtr.DbtrAgt    False
Dbtr.BIC        False
Cdtr.Nm         False
Cdtr.ActNo      False
Cdtr.CdtrAgt    False
Cdtr.BIC        False
NbOfTxs         False
Amt             False
PmtMtd          False
Ccy             False
CreDtTm         False
InstrPrty       False
XchgRateInf      True
XchgRate        False
ChqNb           False
MemoFld         False
Rsn              True
Class           False
dtype: bool

In [396]:
# checking column wise null value count
df.isnull().any()
df.isnull().sum()

PmtInfId           0
Dbtr.Nm            0
Dbtr.ActNo         0
Dbtr.DbtrAgt       0
Dbtr.BIC           0
Cdtr.Nm            0
Cdtr.ActNo         0
Cdtr.CdtrAgt       0
Cdtr.BIC           0
NbOfTxs            0
Amt                0
PmtMtd             0
Ccy                0
CreDtTm            0
InstrPrty          0
XchgRateInf     3172
XchgRate           0
ChqNb              0
MemoFld            0
Rsn             1600
Class              0
dtype: int64

In [397]:
# checking overall null value count
df.isnull().sum().sum()

4772

#  Percentage of missing values

In [398]:

df.isnull().sum()/len(df)*100

PmtInfId         0.000
Dbtr.Nm          0.000
Dbtr.ActNo       0.000
Dbtr.DbtrAgt     0.000
Dbtr.BIC         0.000
Cdtr.Nm          0.000
Cdtr.ActNo       0.000
Cdtr.CdtrAgt     0.000
Cdtr.BIC         0.000
NbOfTxs          0.000
Amt              0.000
PmtMtd           0.000
Ccy              0.000
CreDtTm          0.000
InstrPrty        0.000
XchgRateInf     99.125
XchgRate         0.000
ChqNb            0.000
MemoFld          0.000
Rsn             50.000
Class            0.000
dtype: float64

In [399]:
# assuming XchgRateInf is zero
df["XchgRateInf"].fillna(0, inplace = True)
df

,PmtInfId,Dbtr.Nm,Dbtr.ActNo,Dbtr.DbtrAgt,Dbtr.BIC,Cdtr.Nm,Cdtr.ActNo,Cdtr.CdtrAgt,Cdtr.BIC,NbOfTxs,...,PmtMtd,Ccy,CreDtTm,InstrPrty,XchgRateInf,XchgRate,ChqNb,MemoFld,Rsn,Class
0,Viw,Apple,45831890872,Wells Fargo,WFBIUS6S123,Walmart,2689301692,Capital One,NFBKUSF1123,1,...,ACH,USD,07/10/2021,Low,0.0,1,2,xGip,Mueller LLC,1
1,5L4,Walmart,54698287911,Wells Fargo,WFBIUS6S123,Walmart,5469828791,Wells Fargo,WFBIUS6S123,1,...,ACH,USD,29/09/2021,Medium,0.0,1,8060217,ChCn,NaN,1
2,dae,Apple,45831890872,Wells Fargo,WFBIUS6S123,Meta,482970124330,JPMC,CHASUS33123,1,...,WIRE,USD,22/04/2022,Low,0.0,1,0,psL3,NaN,1
3,mRh,Google,16182937803,Wells Fargo,WFBIUS6S123,Apple,3672937640,Capital One,NFBKUSF1123,1,...,CARD,USD,08/01/2022,High,0.0,1,18,bEHx,NaN,1
4,2KG,Apple,45831890872,Wells Fargo,WFBIUS6S123,Amazon,123456789100,Capital One,NFBKUSF1123,1,...,CARD,USD,23/05/2022,Medium,0.0,1,9143273,fY3c,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,E30,Meta,97631540681,Wells Fargo,WFBIUS6S123,Amazon,39021229011,PJMC,CHASUS33123,1,...,WIRE,USD,18/01/2022,Low,0.0,1,461,HS4j,Deckow Group,4
3196,0xg,Meta,97631540681,Wells Fargo,WFBIUS6S123,Amazon,39021229011,JPM,CHASUS33123,1,...,CARD,USD,19/12/2021,Low,0.0,1,2092,jgQ6,NaN,4
3197,AF1,Apple,45831890872,Wells Fargo,WFBIUS6S123,Walmart,20777256012,Goldman Saachs,GSCMUS33123,1,...,ACH,USD,10/02/2022,High,0.0,1,241,C6lf,Beahan and Sons,4
3198,G5s,Apple,45831890872,Wells Fargo,WFBIUS6S123,Apple,45831890872,Wells Faargo,WFBIUS6S123,1,...,ACH,USD,27/05/2022,Low,0.0,1,7099799,T7n9,Steuber-Howe,4


# 50 Percent of Data is empty we will inpute it with random Text

In [400]:
import string
import random
# initializing size of string 
N = 7
# using random.choices()
# generating random strings 
res = ''.join(random.choices(string.ascii_uppercase +
                             string.digits, k = N))

df["Rsn"].fillna(str(res), inplace = True)
df.head(5)

,PmtInfId,Dbtr.Nm,Dbtr.ActNo,Dbtr.DbtrAgt,Dbtr.BIC,Cdtr.Nm,Cdtr.ActNo,Cdtr.CdtrAgt,Cdtr.BIC,NbOfTxs,...,PmtMtd,Ccy,CreDtTm,InstrPrty,XchgRateInf,XchgRate,ChqNb,MemoFld,Rsn,Class
0,Viw,Apple,45831890872,Wells Fargo,WFBIUS6S123,Walmart,2689301692,Capital One,NFBKUSF1123,1,...,ACH,USD,07/10/2021,Low,0.0,1,2,xGip,Mueller LLC,1
1,5L4,Walmart,54698287911,Wells Fargo,WFBIUS6S123,Walmart,5469828791,Wells Fargo,WFBIUS6S123,1,...,ACH,USD,29/09/2021,Medium,0.0,1,8060217,ChCn,L82GAVL,1
2,dae,Apple,45831890872,Wells Fargo,WFBIUS6S123,Meta,482970124330,JPMC,CHASUS33123,1,...,WIRE,USD,22/04/2022,Low,0.0,1,0,psL3,L82GAVL,1
3,mRh,Google,16182937803,Wells Fargo,WFBIUS6S123,Apple,3672937640,Capital One,NFBKUSF1123,1,...,CARD,USD,08/01/2022,High,0.0,1,18,bEHx,L82GAVL,1
4,2KG,Apple,45831890872,Wells Fargo,WFBIUS6S123,Amazon,123456789100,Capital One,NFBKUSF1123,1,...,CARD,USD,23/05/2022,Medium,0.0,1,9143273,fY3c,L82GAVL,1


In [401]:
# checking overall null value count
df.isnull().sum().sum()

0

# Lets Check for Duplicate

In [402]:
print("Number of duplicate values in the dataset is : ", df.duplicated().sum())

Number of duplicate values in the dataset is :  0


# Now we will Convert Items into Categorical field for Dbtr.Nm assign
1 ->  Apple
2 ->  Walmart
3 -> Google
4 ->  MetaI
5 ->  Amazon

In [403]:

df.loc[df['Dbtr.Nm'] == 'Apple'] = 1
df.loc[df['Dbtr.Nm'] == 'Walmart'] = 2
df.loc[df['Dbtr.Nm'] == 'Google'] = 3
df.loc[df['Dbtr.Nm']== 'Meta'] = 4
df.loc[df['Dbtr.Nm'] == 'Amazon'] = 5
df['Dbtr.Nm'].head(5)


0    1
1    2
2    1
3    3
4    1
Name: Dbtr.Nm, dtype: int64

# Now we will Convert Items into Categorical field for Dbtr.BIC assign
1 ->  WFBIUS6S123


In [404]:
df.loc[df['Dbtr.BIC'] == 'WFBIUS6S123'] = 1
df['Dbtr.BIC'].head(5)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


0    1
1    2
2    1
3    3
4    1
Name: Dbtr.BIC, dtype: int64

# Now we will Convert Items into Categorical field for Dbtr.Nm assign
1 ->  Apple
2 ->  Walmart
3 -> Google
4 ->  MetaI
5 ->  Amazon

In [405]:

df.loc[df['Cdtr.Nm'] == 'Apple'] = 1
df.loc[df['Cdtr.Nm'] == 'Walmart'] = 2
df.loc[df['Cdtr.Nm'] == 'Google'] = 3
df.loc[df['Cdtr.Nm']== 'Meta'] = 4
df.loc[df['Cdtr.Nm'] == 'Amazon'] = 5
df['Cdtr.Nm'].head(5)


0    1
1    2
2    1
3    3
4    1
Name: Cdtr.Nm, dtype: int64

# Now we will Convert Items into Categorical field for Cdtr.BIC assign
1 ->  NFBKUSF1123
2 ->  WFBIUS6S123
3 -> CHASUS33123
4 ->  GSCMUS33123


In [406]:
df.loc[df['Cdtr.BIC'] == 'NFBKUSF1123'] = 1
df.loc[df['Cdtr.BIC'] == 'WFBIUS6S123'] = 2
df.loc[df['Cdtr.BIC'] == 'CHASUS33123'] = 3
df.loc[df['Cdtr.BIC'] == 'GSCMUS33123'] = 4
df['Cdtr.BIC'].head(5)

0    1
1    2
2    1
3    3
4    1
Name: Cdtr.BIC, dtype: int64

# Now we will Convert Items into Categorical field for PmtMtd assign
1 ->  ACH
2 ->  WIRE
3 -> CARD

In [407]:
df.loc[df['PmtMtd'] == 'ACH'] = 1
df.loc[df['PmtMtd'] == 'WIRE'] = 2
df.loc[df['PmtMtd'] == 'CARD'] = 3
df['PmtMtd'].head(5)

0    1
1    2
2    1
3    3
4    1
Name: PmtMtd, dtype: int64

# Now we will Convert Items into Categorical field for InstrPrty assign
1 ->  Low
2 ->  Medium
3 -> High


In [408]:
df.loc[df['InstrPrty'] == 'Low'] = 1
df.loc[df['InstrPrty'] == 'Medium'] = 2
df.loc[df['InstrPrty'] == 'High'] = 3
df['InstrPrty'].head(5)

0    1
1    2
2    1
3    3
4    1
Name: InstrPrty, dtype: int64

# Now we will Convert Items into Categorical field for Ccy assign
1 ->  USD



In [409]:
df.loc[df['Ccy'] == 'USD'] = 1
df['Ccy'].head(5)

0    1
1    2
2    1
3    3
4    1
Name: Ccy, dtype: int64

# Finding correlation

In [410]:
df.corr().style.format("{:.4}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

,PmtInfId,Dbtr.Nm,Dbtr.ActNo,Dbtr.DbtrAgt,Dbtr.BIC,Cdtr.Nm,Cdtr.ActNo,Cdtr.CdtrAgt,Cdtr.BIC,NbOfTxs,Amt,PmtMtd,Ccy,CreDtTm,InstrPrty,XchgRateInf,XchgRate,ChqNb,MemoFld,Rsn,Class
PmtInfId,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Dbtr.Nm,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Dbtr.ActNo,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Dbtr.DbtrAgt,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Dbtr.BIC,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Cdtr.Nm,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Cdtr.ActNo,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Cdtr.CdtrAgt,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Cdtr.BIC,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
NbOfTxs,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [411]:
#sns.pairplot(df, hue="Class")
#plt.show()

# Now Lets See DataFrame

In [412]:
df

,PmtInfId,Dbtr.Nm,Dbtr.ActNo,Dbtr.DbtrAgt,Dbtr.BIC,Cdtr.Nm,Cdtr.ActNo,Cdtr.CdtrAgt,Cdtr.BIC,NbOfTxs,...,PmtMtd,Ccy,CreDtTm,InstrPrty,XchgRateInf,XchgRate,ChqNb,MemoFld,Rsn,Class
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1.0,1,1,1,1,1
1,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2.0,2,2,2,2,2
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1.0,1,1,1,1,1
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3.0,3,3,3,3,3
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1.0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4.0,4,4,4,4,4
3196,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4.0,4,4,4,4,4
3197,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1.0,1,1,1,1,1
3198,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1.0,1,1,1,1,1


# Declare feature vector and target variable

In [413]:
X = df.drop(['Class'], axis=1)

y = df['Class']

# Split data into separate training and test set 

In [414]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [415]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

((2240, 20), (960, 20))

In [416]:
# import Random Forest classifier

from sklearn.ensemble import RandomForestClassifier

# instantiate the classifier 

rfc = RandomForestClassifier(random_state=0)

# fit the model

rfc.fit(X_train, y_train)

# Predict the Test set results

y_pred = rfc.predict(X_test)

# Check accuracy score 

from sklearn.metrics import accuracy_score

print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with 10 decision-trees : 1.0000


In [417]:
# Print the Confusion Matrix and slice it into four pieces

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

print('Confusion matrix\n\n', cm)

Confusion matrix

 [[180   0   0   0   0]
 [  0 208   0   0   0]
 [  0   0 181   0   0]
 [  0   0   0 185   0]
 [  0   0   0   0 206]]


In [418]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       180
           2       1.00      1.00      1.00       208
           3       1.00      1.00      1.00       181
           4       1.00      1.00      1.00       185
           5       1.00      1.00      1.00       206

    accuracy                           1.00       960
   macro avg       1.00      1.00      1.00       960
weighted avg       1.00      1.00      1.00       960



In [419]:
 X = df.drop(['Class'], axis=1)
rfc.predict(X)


array([1, 2, 1, ..., 1, 1, 1], dtype=int64)

In [420]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

# Decision Tree Classifier Hyperparameter tuning

In [421]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators, 'max_features': max_features,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}
rf_clf = RandomForestClassifier(random_state=42)

rf_cv = RandomizedSearchCV(estimator=rf_clf, scoring='f1',param_distributions=random_grid, n_iter=100, cv=3, 
                               verbose=2, random_state=42, n_jobs=-1)

rf_cv.fit(X_train, y_train)
rf_best_params = rf_cv.best_params_
print(f"Best paramters: {rf_best_params})")

rf_clf = RandomForestClassifier(**rf_best_params)
rf_clf.fit(X_train, y_train)

print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\User\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]
  category=UserWarning,


Best paramters: {'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True})
Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
               1      2      3      4      5  accuracy  macro avg  \
precision    1.0    1.0    1.0    1.0    1.0       1.0        1.0   
recall       1.0    1.0    1.0    1.0    1.0       1.0        1.0   
f1-score     1.0    1.0    1.0    1.0    1.0       1.0        1.0   
support    436.0  428.0  399.0  523.0  454.0       1.0     2240.0   

           weighted avg  
precision           1.0  
recall              1.0  
f1-score            1.0  
support          2240.0  
_______________________________________________
Confusion Matrix: 
 [[436   0   0   0   0]
 [  0 428   0   0   0]
 [  0   0 399   0   0]
 [  0   0   0 523   0]
 [  0   0   0   0 454]]

Test Result:
Accuracy Score: 100.00%
_______________________________________________

# Grid Search Cross Validation

In [422]:
from sklearn.model_selection import GridSearchCV
n_estimators = [100, 500, 1000, 1500]
max_features = ['auto', 'sqrt']
max_depth = [2, 3, 5]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4, 10]
bootstrap = [True, False]

params_grid = {'n_estimators': n_estimators, 'max_features': max_features,
               'max_depth': max_depth, 'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}

rf_clf = RandomForestClassifier(random_state=42)

rf_cv = GridSearchCV(rf_clf, params_grid, scoring="f1", cv=3, verbose=2, n_jobs=-1)
rf_cv.fit(X_train, y_train)
best_params = rf_cv.best_params_
print(f"Best parameters: {best_params}")

rf_clf = RandomForestClassifier(**best_params)
rf_clf.fit(X_train, y_train)

print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)

Fitting 3 folds for each of 768 candidates, totalling 2304 fits


C:\Users\User\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan 

Best parameters: {'bootstrap': True, 'max_depth': 2, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
               1      2      3      4      5  accuracy  macro avg  \
precision    1.0    1.0    1.0    1.0    1.0       1.0        1.0   
recall       1.0    1.0    1.0    1.0    1.0       1.0        1.0   
f1-score     1.0    1.0    1.0    1.0    1.0       1.0        1.0   
support    436.0  428.0  399.0  523.0  454.0       1.0     2240.0   

           weighted avg  
precision           1.0  
recall              1.0  
f1-score            1.0  
support          2240.0  
_______________________________________________
Confusion Matrix: 
 [[436   0   0   0   0]
 [  0 428   0   0   0]
 [  0   0 399   0   0]
 [  0   0   0 523   0]
 [  0   0   0   0 454]]

Test Result:
Accuracy Score: 100.00%
_______________________________________________
